In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import torch
from torch import Tensor
import torch.nn as nn
import pandas as pd
import numpy as np

In [20]:
preds = torch.tensor([[0.5, 0.4, 0.3, 0.1],
                      [0.9, 0.2, 0.7, 0.8]])
labels = torch.tensor([[0., 1., 1., 0.],
                       [1., 0., 0., 0.]])

In [32]:
loss = nn.BCELoss(reduction='none')
loss(preds, labels)

tensor([[0.6931, 0.9163, 1.2040, 0.1054],
        [0.1054, 0.2231, 1.2040, 1.6094]])

In [29]:
weight = torch.tensor([[1., 2., 3., 4.]])

In [30]:
loss = nn.BCELoss(weight=weight, reduction='none')

In [35]:
loss(preds, labels)

tensor([[0.6931, 0.9163, 1.2040, 0.1054],
        [0.1054, 0.2231, 1.2040, 1.6094]])

In [34]:
torch.mul(weight, loss(preds, labels))

tensor([[0.6931, 1.8326, 3.6119, 0.4214],
        [0.1054, 0.4463, 3.6119, 6.4378]])